## Assignment 1 Part 2
### Name:
### Roll Number:

In [ ]:
# importing all the necessary libraries
# pandas for reading the csv file, numpy for mathematical operations, matplotlib for plotting the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# extracting the data from the csv file, checking the shape of the data
df = pd.read_csv('../../dataset/decision-tree.csv')
print(df.shape)

# randomizing the dataset
df = df.sample(frac=1).reset_index(drop=True)

print("First 5 rows of the dataset:")
df.head()

(768, 9)
First 5 rows of the dataset:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,155,74,17,96,26.6,0.433,27,1
1,2,90,80,14,55,24.4,0.249,24,0
2,2,109,92,0,0,42.7,0.845,54,0
3,3,182,74,0,0,30.5,0.345,29,1
4,1,108,60,46,178,35.5,0.415,24,0


In [ ]:
# splitting the dataset into training and testing data
# train set: 80%, test set: 20%
train_data = df.iloc[:int(0.8*len(df)), :]
test_data = df.iloc[int(0.8*len(df)):, :]


# check the shape of the data
print("Shape of the training data:", train_data.shape)
print("Shape of the testing data:", test_data.shape)

Shape of the training data: (614, 9)
Shape of the testing data: (154, 9)
